In [1]:
import pandas as pd
import mlflow
import mlflow.sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Simüle veri
df = pd.DataFrame({
    "user_age": [22, 45, 36, 29, 52, 40],
    "country": ["TR", "DE", "FR", "US", "UK", "TR"],
    "is_fraud": [0, 1, 0, 0, 1, 0]
})

# Country -> one-hot encoding
df = pd.get_dummies(df, columns=["country"])

X = df.drop("is_fraud", axis=1)
y = df["is_fraud"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# MLflow deneyi başlat
mlflow.set_experiment("fraud_detection")

with mlflow.start_run():
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)

    preds = model.predict(X_test)
    acc = accuracy_score(y_test, preds)

    mlflow.log_metric("accuracy", acc)
    mlflow.sklearn.log_model(model, "model")
    mlflow.log_params({
        "n_estimators": 100,
        "model_type": "RandomForest"
    })

    print("Accuracy:", acc)


2025/06/22 15:52:47 INFO mlflow.tracking.fluent: Experiment with name 'fraud_detection' does not exist. Creating a new experiment.
2025/06/22 15:52:47 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/06/22 15:52:50 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Accuracy: 0.5


In [2]:
import redis

# Redis bağlantısı (docker-compose'daki adres ve port)
r = redis.Redis(host="localhost", port=6379, decode_responses=True)

entity_id = "123"
features = {
    "user_age": 42,
    "country": "TR"
}

# Özellikleri Redis'e yaz
r.hset(entity_id, mapping=features)

# Doğrulama: Redis'ten oku
print("Redis ->", r.hgetall(entity_id))


Redis -> {'user_age': '42', 'country': 'TR'}
